In [85]:
import requests
import math
import json
import pandas as pd
import re
from sodapy import Socrata
import multiprocessing

In [55]:
def cd():
    data = requests.get('https://data.cityofnewyork.us/api/views/cc5c-sm6z/rows.json')
    raw_data = data.json()['data']
    cleaner_data = []
    for lane in raw_data:
        coords = lane[8].split(',')
        if lane[-1] != None:
            cleaner_data.append({'first_coord': (float(coords[0][18:].split(' ')[1]), float(coords[0][18:].split(' ')[0])), 'second_coord': (float(coords[1][1:-2].split(' ')[1]), float(coords[1][1:-2].split(' ')[0])), 'type': lane[-1]})
        else:
            cleaner_data.append({'first_coord': (float(coords[0][18:].split(' ')[1]), float(coords[0][18:].split(' ')[0])), 'second_coord': (float(coords[1][1:-2].split(' ')[1]), float(coords[1][1:-2].split(' ')[0])), 'type': 'Unknown'})
    return cleaner_data

In [56]:
lane_data = cd()

In [ ]:
def quick_dist(coord1, coord2):
    return math.sqrt(math.pow(12430*((coord1[1]-coord2[1])/180),2)+math.pow(24901*((coord1[0]-coord2[0])/360)*0.16133111759,2))

In [57]:
def cds(data):
    actionable_data = []
    assembly_array = []
    count = 0
    for lane in data:
        newlane = lane
        count += 1
        for option in data:
            if quick_dist(lane['first_coord'], option['first_coord']) < 0.5 and lane['first_coord'] != option['first_coord'] and lane['second_coord'] != option['second_coord']:
                assembly_array.append({'first_coord': option['first_coord'], 'second_coord': option['second_coord'], 'type': option['type']})
        newlane['options'] = assembly_array
        assembly_array = []
        if count % 1000 == 0:
            print(str(int(count/len(data)*100))+'%')
        actionable_data.append(newlane)
    return actionable_data

In [58]:
clean_data = cds(lane_data)

5%
10%
16%
21%
26%
32%
37%
42%
48%
53%
58%
64%
69%
74%
80%
85%
90%
96%


In [86]:
def retrieve_crashes():
    client = Socrata("data.cityofnewyork.us", None)
    results = client.get("h9gi-nx95", limit="100000000")
    return results

In [87]:
crash_data = retrieve_crashes()

In [111]:
def parse_crashes(data):
    regex = r"vehicle_type_code\d+"
    bike_accidents = []
    errors = 0
    for crash in data:
        for item in crash.keys():
            if re.match(regex, item) and crash[item] == 'Bike' and int(crash['number_of_cyclist_injured'] + crash['number_of_cyclist_killed']) > 0:
                try:
                    bike_accidents.append({
                        'date': crash['crash_date'],
                        'time': crash['crash_time'],
                        'latitude': float(crash['latitude']),
                        'longitude': float(crash['longitude']),
                        'number_of_cyclist_injured': crash['number_of_cyclist_injured'],
                        'number_of_cyclist_killed': crash['number_of_cyclist_killed'],
                    })
                except:
                    errors += 1
                break
    print("The percentage of bike crashes which are invalid is:", errors/len(bike_accidents))
    return bike_accidents

In [112]:
result = parse_crashes(crash_data)

The percentage of bike crashes which are invalid is: 0.08203823035685276


In [77]:
def score_lanes(crash_data, lane_data):
    for lane in lane_data:
        lane_dist = quick_dist(lane['first_coord'], lane['second_coord']) / 2
        for crash in crash_data:
            first_dist = quick_dist(lane['first_coord'], (crash['latitude'], crash['longitude']))
            second_dist = quick_dist(lane['second_coord'], (crash['latitude'], crash['longitude']))
            if lane_dist > first_dist or lane_dist > second_dist:
                try:
                    lane['injuries'] += crash['number_of_cyclist_injured']
                    lane['deaths'] += crash['number_of_cyclist_killed']
                except:
                    lane['injuries'] = crash['number_of_cyclist_injured']
                    lane['deaths'] = crash['number_of_cyclist_killed']
    return lane_data

In [78]:
updated_lanes = score_lanes(result, clean_data)

KeyboardInterrupt: 

In [110]:
len(result)

24315

In [82]:
len(clean_data)

18730

In [117]:
result[-1]

{'date': '2012-09-12T00:00:00.000',
 'time': '21:10',
 'latitude': 40.63659,
 'longitude': -73.994736,
 'number_of_cyclist_injured': '1',
 'number_of_cyclist_killed': '0'}